In [1]:
from pymilvus import MilvusClient
from pymilvus import CollectionSchema, FieldSchema, DataType


import pandas as pd
from tqdm import tqdm
import numpy as np
from typing import Dict , List, Any
import time
import json



from FlagEmbedding import BGEM3FlagModel
embeddings_dim = 1024


### Create Collection

In [62]:
client.list_collections()

['articles_collection_L2',
 'articles_collectionPartition_IP',
 'articles_collectionPartition_COSINE',
 'articles_collection_IP',
 'articles_collection_COSINE',
 'articles_collectionPartition_L2']

In [61]:
# client.drop_collection("articles_collection")

In [63]:
client = MilvusClient(uri="http://localhost:19530")

In [64]:

collection_name = 'articles_collection'

id_field = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True)
text_field = FieldSchema(name="article", dtype=DataType.VARCHAR, max_length=65535)
reference_field = FieldSchema(name="reference", dtype=DataType.VARCHAR, max_length=1000)
embedding_field = FieldSchema(name="embedding_articles", dtype=DataType.FLOAT16_VECTOR, dim=embeddings_dim)

schema = CollectionSchema(fields=[id_field, text_field, reference_field, embedding_field], description="collection d'articles de loi")

client.create_collection(collection_name = collection_name, schema=schema)

##### Prep Indexing

In [65]:
#  set the indexing
index_params = MilvusClient.prepare_index_params()

# Add an index on the vector field.
index_params.add_index(
    field_name="embedding_articles",
    metric_type="COSINE",
    index_type="FLAT",
    index_name="vector_index",
    params={}   #128 clusters units, 8 cluster search
)

client.create_index(
    collection_name="articles_collection",
    index_params=index_params,
    sync=False # Whether to wait for index creation to complete before returning. Defaults to True.
)


### Create Partition and Insert entities

##### Create Partitions

In [66]:
# load data
df_articles = pd.read_csv('articles.csv')

Create new categories for partitions

In [67]:
import pandas as pd

# Define a mapping of codes to big categories
code_to_category = {
    "Code Réglementaire Wallon de l'Action sociale et de la Santé": "Protection Sociale et Santé",
    "Code Judiciaire": "Gouvernance et Législation",
    "Code de Droit Economique": "Gouvernance et Législation",
    "Code Civil": "Gouvernance et Législation",
    "Code du Bien-être au Travail": "Protection Sociale et Santé",
    "Code des Sociétés et des Associations": "Commerce et Affaires",
    "Code de la Démocratie Locale et de la Décentralisation": "Administration Locale et Démocratie",
    "Code Wallon de l'Action sociale et de la Santé": "Protection Sociale et Santé",
    "Code de la Navigation": "Environnement et Infrastructures",
    "Code de l'Eau intégré au Code Wallon de l'Environnement": "Environnement et Infrastructures",
    "Code Wallon du Développement Territorial": "Environnement et Infrastructures",
    "Code d'Instruction Criminelle": "Gouvernance et Législation",
    "Code Pénal": "Gouvernance et Législation",
    "Code de la Fonction Publique Wallonne": "Administration Publique",
    "Code Wallon de l'Agriculture": "Environnement et Infrastructures",
    "Code Bruxellois de l'Aménagement du Territoire": "Environnement et Infrastructures",
    "Code Wallon de l'Environnement": "Environnement et Infrastructures",
    "Code Wallon de l'Enseignement Fondamental et de l'Enseignement Secondaire": "Éducation",
    "Code Pénal Social": "Protection Sociale et Santé",
    "Code Wallon de l'Habitation Durable": "Logement et Développement Urbain",
    "Code Bruxellois du Logement": "Logement et Développement Urbain",
    "Code Forestier": "Environnement et Infrastructures",
    "Code Ferroviaire": "Environnement et Infrastructures",
    "Code Electoral": "Administration Locale et Démocratie",
    "Code Bruxellois de l'Air, du Climat et de la Maîtrise de l'Energie": "Environnement et Infrastructures",
    "La Constitution": "Gouvernance et Législation",
    "Codes des Droits et Taxes Divers": "Impôts et Finance",
    "Code de Droit International Privé": "Gouvernance et Législation",
    "Code Wallon du Bien-être des animaux": "Protection Sociale et Santé",
    "Code Electoral Communal Bruxellois": "Administration Locale et Démocratie",
    "Code Consulaire": "Gouvernance et Législation",
    "Code Rural": "Agriculture et Affaires Rurales",
    "Code Pénal Militaire": "Gouvernance et Législation",
    "Code de la Nationalité Belge": "Gouvernance et Législation"
}

# Map the 'code' column to the new categories
df_articles['big_category'] = df_articles['code'].map(code_to_category)

In [68]:
import re

def normalize_partition_name(name):
    # Replace the spaces and dashes with underscores
    name = re.sub(r'\s+|-', '_', name)
    # remove non-alphanumeric characters
    name = re.sub(r'[^a-zA-Z0-9_]', '', name)
    return name

# Appliquer cette fonction aux noms des codes
df_articles['normalized_cat'] = df_articles['big_category'].apply(normalize_partition_name)


In [70]:
codes

array(['Environnement_et_Infrastructures',
       'Logement_et_Dveloppement_Urbain', 'Gouvernance_et_Lgislation',
       'Administration_Locale_et_Dmocratie', 'Protection_Sociale_et_Sant',
       'Agriculture_et_Affaires_Rurales', 'ducation',
       'Administration_Publique', 'Commerce_et_Affaires',
       'Impts_et_Finance'], dtype=object)

In [69]:
# create the partitions
codes = df_articles['normalized_cat'].unique()

for code in codes:
    client.create_partition(collection_name="articles_collection",
                            partition_name=code)

##### Insert the entities

In [71]:
def load_embeddings_from_file(filepath: str) -> Dict[int, np.ndarray]:
    # Load the npz file
    data = np.load(filepath)
    
    # Convert arrays back to dictionary
    embeddings_dict = {
        int(id_): emb for id_, emb in zip(data['ids'], data['embeddings']) 
    }
    
    print(f"Successfully loaded {len(embeddings_dict)} embeddings")
    
    return embeddings_dict
        

In [72]:
# load the embeddings
embeddings_dict = load_embeddings_from_file('embeddings_bgem3.npz')

Successfully loaded 22633 embeddings


In [73]:
# Create the entities
def process_embeddings_in_chunks_and_partitions(
    df_articles: pd.DataFrame,
    loaded_embeddings: Dict[int, np.ndarray],
    chunk_size: int = 1000
) -> Dict[str, List[Dict[str, Any]]]:

    # Initialize dictionary for partitioned entities
    partitioned_entities = {}

    # Process data in chunks with progress bar
    with tqdm(total=len(df_articles), desc="Processing entities") as pbar:
        for chunk_start in range(0, len(df_articles), chunk_size):
            # Get chunk of dataframe
            chunk_end = min(chunk_start + chunk_size, len(df_articles))
            df_chunk = df_articles.iloc[chunk_start:chunk_end]

            # Process chunk
            for _, row in df_chunk.iterrows():
                partition = row['normalized_cat']
                embedding = loaded_embeddings.get(row['id'])
                if embedding is not None:
                    entity = {
                        "id": row['id'],
                        "article": row['article'],
                        "reference": row['reference'],
                        "embedding_articles": np.array(embedding, dtype=np.float16)  # np.array of float16 to meet Milvus requirements
                    }
                    if partition not in partitioned_entities:
                        partitioned_entities[partition] = []
                    partitioned_entities[partition].append(entity)

            # Update progress bar
            pbar.update(len(df_chunk))

    print(f"Processed entities into {len(partitioned_entities)} partitions.")
    return partitioned_entities


In [74]:
partition_entities = process_embeddings_in_chunks_and_partitions(df_articles, embeddings_dict)

Processing entities: 100%|██████████| 22633/22633 [00:01<00:00, 13273.58it/s]


Processed entities into 10 partitions.


In [75]:
def insert_entities_in_chunks(client, collection_name, partition_entities, batch_size=1000):
    for partition, entities in partition_entities.items():
        print(f"Inserting entities into partition: {partition}")
        for i in range(0, len(entities), batch_size):
            batch = entities[i:i + batch_size]
            try:
                client.insert(data=batch, collection_name=collection_name, partition_name=partition)
            except Exception as e:
                print(f"Error during insertion for batch {i // batch_size + 1} in partition {partition}: {e}")



In [76]:
# Example usage
insert_entities_in_chunks(client, collection_name, partition_entities, batch_size=1000)

Inserting entities into partition: Environnement_et_Infrastructures
Inserting entities into partition: Logement_et_Dveloppement_Urbain
Inserting entities into partition: Gouvernance_et_Lgislation
Inserting entities into partition: Administration_Locale_et_Dmocratie
Inserting entities into partition: Protection_Sociale_et_Sant
Inserting entities into partition: Agriculture_et_Affaires_Rurales
Inserting entities into partition: ducation
Inserting entities into partition: Administration_Publique
Inserting entities into partition: Commerce_et_Affaires
Inserting entities into partition: Impts_et_Finance


-------------

### Test search: Retrieve

In [136]:
res = client.list_partitions(collection_name="articles_collection")
print(res)

['_default', 'Environnement_et_Infrastructures', 'Logement_et_Dveloppement_Urbain', 'Gouvernance_et_Lgislation', 'Administration_Locale_et_Dmocratie', 'Protection_Sociale_et_Sant', 'Agriculture_et_Affaires_Rurales', 'ducation', 'Administration_Publique', 'Commerce_et_Affaires', 'Impts_et_Finance']


##### Embedding Model

In [78]:
bge_m3 = BGEM3FlagModel('BAAI/bge-m3',  
                       use_fp16=True, 
                       device='cuda')


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

c:\Users\mthia\anaconda3\envs\AI_App\Lib\site-packages\FlagEmbedding\BGE_M3\modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  colbert_state_dict = torch.load(os

In [79]:
def generate_embedding(article):
    embedding = bge_m3.encode([article], batch_size=12, max_length=8*1024)["dense_vecs"]
    return embedding[0]

In [111]:
df_questions = pd.read_csv('questions_train.csv')


df_questions['complet_question'] = df_questions.apply(lambda row: row['question'] if pd.isna(row['extra_description']) else row['question'] + " " + row['extra_description'], axis=1)

In [117]:
# Define the mapping from mid-categories to big categories
mid_to_big_category = {
    "Famille": ["Protection Sociale et Santé", "Éducation"],
    "Logement": ["Logement et Développement Urbain", "Environnement et Infrastructures"],
    "Argent": ["Commerce et Affaires", "Impôts et Finance"],
    "Justice": ["Gouvernance et Législation", "Administration Locale et Démocratie"],
    "Etrangers": ["Gouvernance et Législation", "Protection Sociale et Santé"],
    "Protection sociale": ["Protection Sociale et Santé", "Gouvernance et Législation"],
    "Travail": ["Protection Sociale et Santé", "Commerce et Affaires"]
}

# Function to map mid-category to big categories
def map_to_big_category(mid_cat):
    return mid_to_big_category.get(mid_cat, ["Unknown"])

# Apply the mapping to create a new column
df_questions['big_category'] = df_questions['category'].apply(map_to_big_category)



In [86]:
m = 3
query_vectors = generate_embedding(df_questions['complet_quesiton'].iloc[m])

In [27]:
codes

array(['Environnement_et_Infrastructures',
       'Logement_et_Dveloppement_Urbain', 'Gouvernance_et_Lgislation',
       'Administration_Locale_et_Dmocratie', 'Protection_Sociale_et_Sant',
       'Agriculture_et_Affaires_Rurales', 'ducation',
       'Administration_Publique', 'Commerce_et_Affaires',
       'Impts_et_Finance'], dtype=object)

In [88]:
# selected_partitions = ["Protection_Sociale_et_Sant"]
selected_partitions =  df_questions['big_category'].iloc[m]

normalized_selected_partitions = [re.sub(r'\s+|-', '_', partition) for partition in selected_partitions]
normalized_selected_partitions = [re.sub(r'[^a-zA-Z0-9_]', '', partition) for partition in normalized_selected_partitions]



print(selected_partitions)
print(normalized_selected_partitions)

['Logement et Développement Urbain', 'Environnement et Infrastructures']
['Logement_et_Dveloppement_Urbain', 'Environnement_et_Infrastructures']


In [122]:
client.load_partitions(collection_name = collection_name,
                                    partition_names= normalized_selected_partitions)

search_results = client.search(
    collection_name= collection_name,
    data = [query_vectors],  
    partition_names= normalized_selected_partitions,
    limit=20,  
    search_params={"metric_type": "COSINE", "params":{}},
    output_fields= ['id', 'article']
)

In [92]:
print(df_questions['article_ids'].iloc[m])
formatted_result = json.dumps(search_results[0], indent=3, ensure_ascii=False)
print(formatted_result)

12012,12030,12031,12032,12033,12034,12035
[
   {
      "id": 12154,
      "distance": 0.6632996797561646,
      "entity": {
         "id": 12154,
         "article": "La société doit procéder au recrutement d'un référent social chargé d'assurer le lien avec les acteurs sociaux locaux pour offrir un accompagnement social aux locataires de logements d'utilité publique gérés par la société, en veillant particulièrement à ce que l'accompagnement des personnes en transition entre les modes d'hébergement et le logement social accompagné soit assuré.Le Gouvernement détermine les conditions de recrutement du référent social ainsi que les modalités de mise en réseau de l'accompagnement social.Le Gouvernement subventionne la rémunération du référent social dans les conditions qu'il détermine."
      }
   },
   {
      "id": 12053,
      "distance": 0.6588654518127441,
      "entity": {
         "id": 12053,
         "article": "Le Gouvernementpeut accorder une aide à toute société de logement de

-----------

In [124]:
entity_ids = [result["entity"]["article"] for result in search_results[0]]
print(entity_ids)

["La société doit procéder au recrutement d'un référent social chargé d'assurer le lien avec les acteurs sociaux locaux pour offrir un accompagnement social aux locataires de logements d'utilité publique gérés par la société, en veillant particulièrement à ce que l'accompagnement des personnes en transition entre les modes d'hébergement et le logement social accompagné soit assuré.Le Gouvernement détermine les conditions de recrutement du référent social ainsi que les modalités de mise en réseau de l'accompagnement social.Le Gouvernement subventionne la rémunération du référent social dans les conditions qu'il détermine.", "Le Gouvernementpeut accorder une aide à toute société de logement de service public qui acquiert un bâtiment non améliorable en vue de le démolir et d'affecter le terrain ainsi libéré à la construction de logements, et accessoirement, dans les limites fixées par le Gouvernement, à des équipements d'intérêt collectif en ce compris les éléments constitutifs d'un résea

#### ALL Predictions

In [137]:
def retrieve_all_predictions(df_questions, client, collection_name):
    predictions_dict = {}

    # all partitions
    all_partitions = client.list_partitions(collection_name=collection_name)

    for m in tqdm(range(len(df_questions))):
        # Generate the query vectors
        query_vectors = generate_embedding(df_questions['complet_question'].iloc[m])
        
        # # Select and normalize partitions
        # selected_partitions = df_questions['big_category'].iloc[m]
        # normalized_selected_partitions = [re.sub(r'\s+|-', '_', partition) for partition in selected_partitions]
        # normalized_selected_partitions = [re.sub(r'[^a-zA-Z0-9_]', '', partition) for partition in normalized_selected_partitions]

        # Load partitions into the client
        client.load_partitions(collection_name=collection_name, partition_names= all_partitions[1:]) # remove the default partition
        
        # Search for results in the collection
        search_results = client.search(
            collection_name=collection_name,
            data=[query_vectors],
            partition_names= all_partitions[1:],
            limit=20,
            search_params={"metric_type": "COSINE", "params": {}},
            output_fields=['id', 'article']
        )
        
        # Collect predicted_ids and predictions_articles
        predicted_ids = [result["entity"]["id"] for result in search_results[0]]
        predictions_articles = [result["entity"]["article"] for result in search_results[0]]
        
        # Populate the dictionary
        predictions_dict[m] = {
            'predicted_ids': predicted_ids,
            'predictions_articles': predictions_articles
        }
    
    return predictions_dict


In [138]:
all_predicted_dict = retrieve_all_predictions(df_questions, client, collection_name)

100%|██████████| 886/886 [01:06<00:00, 13.40it/s]


In [135]:
all_predicted_dict[885]

{'predicted_ids': [20121,
  20276,
  19877,
  20419,
  22257,
  19886,
  20748,
  19911,
  22355,
  20774,
  20664,
  19917,
  22239,
  22391,
  20427,
  19856,
  19858,
  20129,
  20119,
  22241],
 'predictions_articles': ["§ 1er. Les conditions d'émission ou l'assemblée générale des obligataires peuvent désigner un ou plusieurs représentants des obligataires faisant partie de la même émission ou du même programme d'émission. Dans les limites des articles 1984 à 2010 du Code civil, ces représentants peuvent engager tous les obligataires de cette émission ou de ce programme d'émission à l'égard de tiers. Ils peuvent notamment représenter les obligataires dans les procédures d'insolvabilité, en cas de saisie ou dans tout autre cas de concours, dans lequel ils interviennent en leur nom mais pour le compte des obligataires, sans divulguer l'identité de ceux-ci.§ 2. Les conditions d'émission ou l'assemblée générale des obligataires peuvent prévoir en outre que ce représentant intervient ég

In [139]:
import gzip

# Store the result in a compressed JSON file
with gzip.open('all_predictions_collectionALL.json.gz', 'wt', encoding='utf-8') as f:
    json.dump(all_predicted_dict, f)

print('Predictions saved in compressed format!')


Predictions saved in compressed format!
